## Solving bidomain equations on the slice with the Beeler-Reuter model

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-tkiva


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = heart_slice()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 40%] Meshing curve 2 (Nurb)
Info    : [ 70%] Meshing curve 3 (Nurb)
Info    : Done meshing 1D (Wall 0.0447954s, CPU 0.038664s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.068191s, CPU 0.066419s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 2.7654e-05s, CPU 1.4e-05s)
Info    : 3628 nodes 7231 elements


In [3]:
class CellModel(BeelerReuter):
    def applied_current(self):
        def value(x):
            I_app_value = 10
            R1, R2, R3 = 0.2, 0.2, 0.1
            return I_app_value * (
                np.exp(-(((x[0] - 0.1) / R1) ** 2) - ((x[1] - 2.6) / R1) ** 2)
                + np.exp(-(((x[0] - 1.6) / R2) ** 2) - ((x[1] - 1.5) / R2) ** 2)
                + np.exp(-(((x[0] - 3.3) / R3) ** 2) - ((x[1] - 3.25) / R3) ** 2)
            )

        self.I_app.interpolate(value)
        self.I_app_duration = 20
        return self.I_app

In [20]:
class Model(BidomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -85

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(self.x[1] - y_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
                (self.x[0] - x_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
            ]
        )

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EL - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

    def ischemia(self):
        x_c, y_c = 4.6, 1.5
        a, b = 1.0, 0.5

        def reduce(x, reduce_factor: float = 2):
            return 1 + (reduce_factor - 1) * ufl.exp(
                (-(((x[0] - x_c) / a) ** 2) - ((x[1] - y_c) / b) ** 2) * ufl.ln(10)
            )

        def value(x, reduce_factor: float = 2):
            return 1 / (
                1
                + (reduce_factor - 1)
                * np.exp(
                    (-(((x[0] - x_c) / a) ** 2) - ((x[1] - y_c) / b) ** 2) * np.log(10)
                )
            )

        self.M_i = self.M_i / reduce(self.x)
        self.M_e = self.M_e / reduce(self.x)

        fun = fem.Function(self.V1)
        fun.interpolate(value)
        return fun

In [21]:
cell_model = CellModel(domain)
model = Model(domain, cell_model)

In [22]:
plot_function(
    cell_model.I_app,
    camera_direction="xy",
    cmap="RdBu",
    clim = [0,10],
    points=[4.3,0.1,0.0],
    save_to="ischemia_applied_current.pdf",
)

Widget(value='<iframe src="http://localhost:40313/index.html?ui=P_0x7fa4e2fa77d0_5&reconnect=auto" class="pyvi…

In [24]:
plot_function(
    model.ischemia(),
    camera_direction="xy",
    cmap="RdYlGn",
    clim = [1/2,1],
    points=[4.3,0.1,0.0],
    save_to="ischemia_ischemia_location.pdf",
)


Widget(value='<iframe src="http://localhost:40313/index.html?ui=P_0x7fa4c419c310_7&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [8]:
model.solve(
    T=400,
    steps=10000,
    signal_point=[4.3,0.1,0.0],
    save_to="ischemia.mp4",
    camera_direction="xy",
    checkpoints=[35,50,85,100,230,300,350,400],
    checkpoint_file='ischemia'
)

Solving problem: 100%|██████████| 10000/10000 [16:09:08<00:00,  5.81s/it]        


### Plotting fiber orientations

In [9]:
x_c, y_c = 0.2, 1
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.03,
    0.3,
    camera_direction='xy',
    save_to='comparison_fibers.pdf'
)

Widget(value='<iframe src="http://localhost:41721/index.html?ui=P_0x7f993811d6d0_2&reconnect=auto" class="pyvi…

### Saving a signal to a file

In [10]:
np.save('ischemia_signal.npy', model.signal)
np.save('time.npy', model.time)